# The Strength Model
This code uses the Holmedal model to calculate the total yield strength of 6xxx aluminium alloys.
This strength model is here used on alloy conditions from the  SumAl project. See explanatory comments throughout the code.
In this notebook the alloy composition of the alloys in the SumAl project is given in the code, but can easily be changed to contain other alloys as well. A Dark field, length and cross section statistics file is read by the program. The format of the of these .csv files can be found in the "examples" folder. One can also easily substitute the implementation to give the preceipitate distribution and statistics in another way. Lastly there is a thickness excel file that is also read by the program. The file can be found in the "examples" folder. This file contains the corresponding thickness of the sample contained in the dark field images for the alloys. This is needed to correctly calculate the number density of precipitates.

This code uses the Holmedal model to calculate the total yield strength of 6xxx aluminium alloys.
It is applied here to alloy conditions from the SumAl project. See the explanatory comments throughout the code.

In this notebook, the alloy compositions from the SumAl project are defined directly in the code but can easily be modified to include other alloys.

The program reads .csv files containing statistics from dark field images, including precipitate length and cross section. The expected file format can be found in the "examples" folder. The implementation can also be adapted to accept precipitate distributions and statistics from other sources.

The program also reads a thickness file in Excel format (.xlsx), which is stored in the "examples" folder. This file contains the sample thicknesses corresponding to the dark field images for each alloy. These values are required to accurately calculate the number density of precipitates.


In [166]:
#%pip install openpyxl
import numpy as np
import matplotlib.pyplot as plt
import json
from scipy.optimize import least_squares
from scipy import special
from scipy.integrate import quad
import pandas as pd






Constants

In [167]:
# -----------------------------------------------------------------------------
# Obstacle Strength Exponent (kappa)
# -----------------------------------------------------------------------------
# The parameter kappa (κ) describes how the pinning force of shearable
# precipitates evolves with increasing precipitate size. Following Hell et al.
# (2023), we assume that the pinning force scales with cross-sectional area,
# and set κ = 1. This defines how quickly the obstacle strength f approaches 1
# as the precipitate grows beyond the critical cross-section a_c.
# -----------------------------------------------------------------------------
kappa = 1.0  # Obstacle strength exponent, consistent with Hell 2023

M = 2.7         # Taylor factor
G = 27E3        # [MPa] Shear modulus Al matrix
b = 0.286       # [nm] Burgers vector Al, b = a/sqrt(2)
#t = 49.52      # var4_DAA [nm] Sample thickness, HAS TO BE GIVEN.
#t = 38.88       # var7_NA
# -----------------------------------------------------------------------------
# Precipitate Scaling Factor (alpha_p)
# -----------------------------------------------------------------------------
# The parameter alpha_p is a dimensionless scaling factor in the Holmedal model
# that accounts for non-ideal interactions between dislocations and precipitates.

# Following the approach in Hell 2023, we calibrate alpha_p for one known alloy
# condition to match the model-predicted yield strength to
# the corresponding experimental value. This calibrated value is then kept constant
# and used across all other alloy variants and aging conditions to ensure consistency.
#
# Reference: Hell et al., 2023, Equation (11) and Discussion section
# -----------------------------------------------------------------------------
alpha_p = 0.57  # Calibrated scaling factor from one known condition (var6_NA_5h185C), should be around 0.6 from Hell 2023





# Load Data
First run the percipitate statistics models

In [168]:
lengths_file = "var7_PB_statistics_length.csv"
df_length = pd.read_csv(lengths_file)
length_col = [col for col in df_length.columns if "Length" in col]
precipitate_lengths = df_length[length_col[0]].dropna().tolist()
mean_col_l = [col for col in df_length.columns if "Average" in col]
mean_length = df_length[mean_col_l[0]].dropna().iloc[0]
print(len(precipitate_lengths))
print(mean_length)


cross_section_file = "var7_PB_statistics_cross.csv"
df_cross = pd.read_csv(cross_section_file)
cross_col = [col for col in df_cross.columns if "Cross section" in col]
precipitate_cross = df_cross[cross_col[0]].dropna().tolist()
mean_col_c = [col for col in df_cross.columns if "Average" in col]
mean_cross = df_cross[mean_col_c[0]].dropna().iloc[0]
print(len(precipitate_cross))
means_per_image_col = [col for col in df_cross.columns if "Mean per image" in col]
means_per_image = df_cross[means_per_image_col[0]].dropna().tolist()



dark_field_file = "var7_PB_statistics_df.csv"
df_cross_dark = pd.read_csv(dark_field_file)
density_col = [col for col in df_cross_dark.columns if "Number Density [nm^-2]" in col]
number_density = df_cross_dark[density_col[0]].dropna().tolist()

thickness_path = "thickness_samples.xlsx"
df = pd.read_excel(thickness_path)

if 'NA' in dark_field_file or 'PB' in dark_field_file:
    target_header = dark_field_file[:7]
else:
    target_header = dark_field_file[:8]

print(target_header)


thickness_col = [col for col in df.columns if target_header in col]
thickness_array = df[thickness_col[0]].dropna().to_numpy(dtype=float)
print(thickness_array)
print(number_density)

# -----------------------------------------------------------------------------
# Critical Cross-Section Area (a_c)
# -----------------------------------------------------------------------------

# We set the critical cross section area to the mean cross section if our measurede cross
# sections since we are always calculating the strength of peak aged conditions.
# This is in in alignment with Hell 2023 where he smallest measured 𝑎_𝑚𝑒𝑎𝑛 found at peak
# strength is used in the calculations.

# Reference: Hell et al., 2023, Table 6
# -----------------------------------------------------------------------------
crit_cs = np.min(means_per_image)  # [nm²] critical cross-section from Hell 2023
print("Critical Cross Section [nm]: ", crit_cs)



758
18.05
355
var7_PB
[69.20576841 71.05011493 62.8381893  66.37877226 80.7729736  66.97615236
 63.07870895 65.75766951 65.05185366 62.07801029]
[0.0019698, 0.0026636, 0.0022782, 0.0026122, 0.0026293, 0.0024923, 0.0023296, 0.0021925, 0.0021069, 0.0021754]
Critical Cross Section [nm]:  4.9



Functions

In [169]:
def convert_2D_to_3D_number_density(N_per_A, t, mean_length):
    """
    Converts 2D number density (precipitates per nm²) into 3D number density (precipitates per nm³).

    Parameters:
    N_per_A (float): Precipitate count per unit area [#/nm²] for each image of an alloy conditon.
    t (float): Sample thickness [nm] of each image.
    l_mean (float): Mean precipitate length [nm].

    Returns:
    float: 3D number density [#/nm³].
    """
    density = []
    for i in range(len(t)):
        value = (3 * N_per_A[i]) / (t[i] * (1 + (mean_length / t[i])))
        density.append(value)


    return np.mean(np.array(density))

def calculate_volume_fraction(rho, mean_cross, mean_length):
    """
    Computes the precipitate volume fraction.

    Parameters:
    rho (float): Precipitate number density [#/nm³].
    a_mean (float): Mean precipitate cross-sectional area [nm²].
    l_mean (float): Mean precipitate length [nm].

    Returns:
    float: Volume fraction f_V in precentage.
    """
    return rho * mean_cross * mean_length



def omega(x: list, l):
    """
    Computes the aspect ratio function, modeled using a power law:

        Ω(l) = a * l^b

    where:
        - Ω(l) is the aspect ratio of the precipitate.
        - 'a' (x[0]) is the scaling factor.
        - 'b' (x[1]) is the exponent.
        - 'l' is the precipitate length (scalar or array).

    The theoretical aspect ratio for a cylindrical precipitate is:

        Ω = l / sqrt(A)

    Instead of using this directly, we approximate the aspect ratio using a power-law function.

    To avoid values below 1, we enforce Ω(l) ≥ 1.
    """
    arr = x[0] * np.power(l, x[1])  # Compute the power law function

    # Ensure all values in arr are at least 1
    if isinstance(arr, np.ndarray):  
        arr[arr < 1] = 1  
    else:  
        arr = max(arr, 1)  # Ensures scalars do not fall below 1

    return arr

def fit_omega(l, aspect_ratio):
    """
    Finds the best fit for the aspect ratio function Ω(l) = a * l^b.
    
    l: Array of measured precipitate lengths
    aspect_ratio: Corresponding measured aspect ratios (l / sqrt(A))?

    Returns optimized values of a and b.
    """
    x0_fit = [0.7, 0.7]  # Initial guess for parameters (a, b)
    residual = lambda x: aspect_ratio - omega(x, l)  # Residual function
    result = least_squares(residual, x0_fit)  # Perform least-squares optimization
    return result.x  # Return fitted parameters

def tau_p(alpha_p, G, b, n_p, f_bar):
    """
    Strength contribution from precipitate-based obstacles.

    Input:
        alpha_p : Scaling factor
        G       : Shear modulus [MPa]
        b       : Burgers vector [nm]
        n_p     : Number density of precipitate-based obstacles per slip plane [#/nm²]
        f_bar   : Mean obstacle length (dimensionless)

    Output:
        tau_p   : Strength contribution from obstacles [MPa]
    """
    return alpha_p * G * b * np.sqrt(n_p) * f_bar**(3/2) * (1 - 1/6 * f_bar**5)

def phi_tilde(l: float, lengths_data: float, h: float):
    """
    Uncorrected distribution of precipitate lengths.

    Input:
        l  : The length interval to evaluate the distribution of lengths at
        lengths_data : Precipitate length data used to fit the distribution
        h  : Kernel bandwidth for smoothing, determined using Scott's rule

    Output:
        Probability density at length l (before correction)
    """
    return (1 / len(lengths_data)) * np.sum(
    np.sqrt(2) * np.exp(-0.5 * ((np.full(len(lengths_data), l) - np.array(lengths_data)) / h) ** 2) /
    ((1 + special.erf(np.array(lengths_data) / (np.sqrt(2) * h))) * h * np.sqrt(np.pi))
    )



def phi(l, length_data, h):
    """
    Normalized precipitate length distribution.

    This function corrects the unnormalized kernel density estimate 
    phi_tilde, ensuring that the precipitate length distribution meets 
    the required boundary condition, reaching zero at l = 0.

    Input:
        l           : The length interval to evaluate the distribution of lengths at
        length_data : Array of measured precipitate lengths
        h           : Kernel bandwidth for smoothing, determined using Scott's rule

    Output:
        phi         : Normalized probability density of precipitate lengths [1/nm]

    """
    return (phi_tilde(l, length_data, h) - phi_tilde(0, length_data, h) * np.exp(-0.5 * (l / h) ** 2)) / \
           (1 - 0.5 * h * np.sqrt(2 * np.pi) * phi_tilde(0, length_data, h))




def calculated_kernel_bandwidth(length_data):
    """
    Computes the kernel bandwidth (h) for a single experimental condition 
    using Scott's rule:

        h ≈ d * N_l^(-0.2) * sigma_l

    where:
        - h      : Kernel bandwidth for KDE smoothing [nm]
        - d      : Empirical scaling factor (0.8)
        - N_l    : Number of measured precipitate lengths
        - sigma_l: Standard deviation of precipitate lengths

    Input:
        length_data : List or array of measured precipitate length values.

    Output:
        h          : Calculated bandwidth (h) for the given condition.
    """
    sigma = np.std(length_data)  # Compute standard deviation (σ_l)
    N_l = len(length_data)       # Get number of data points (N_l)

    # Apply Scott's rule: h = d * N_l^(-0.2) * σ_l
    h = 0.8 * N_l**(-0.2) * sigma

    return h  # Return the bandwidth value



def shearable_precipitate_integrand(l, aspect_ratio_params, kappa, length_distribution):
    """
    Computes the integrand for the shearable precipitate contribution.

    Based on the integral term:
        ∫ (l^(2κ+1) / Ω(l)^(2κ)) * φ(l) dl

    where:
        - l                   : Precipitate length [nm]
        - aspect_ratio_params  : Parameters for the aspect ratio function Ω(l)
        - kappa               : Scaling exponent for strength model
        - length_distribution : Normalized precipitate length distribution φ(l)

    Output:
        Contribution of shearable precipitates to the mean obstacle strength.
    """
    return (l**(2*kappa + 1) / omega(aspect_ratio_params, l)**(2*kappa)) * length_distribution(l)

def non_shearable_precipitate_integrand(l, length_distribution):
    """
    Computes the integrand for the non-shearable precipitate contribution.

    Based on the integral term:
        ∫ l * φ(l) dl

    where:
        - l                   : Precipitate length [nm]
        - length_distribution : Normalized precipitate length distribution φ(l)

    Output:
        Contribution of non-shearable precipitates to the mean obstacle strength.
    """
    return l * length_distribution(l)


def convert_weight_to_atomic_fraction(weight_percent, atomic_weights, element):
    """
    Convert weight percent (wt%) to atomic fraction (at%).

    Input:
        weight_percent  : Dictionary containing element weight fractions {Element: wt%}
        atomic_weights  : Dictionary containing atomic weights {Element: atomic weight}
        element         : Element to convert

    Output:
        atomic_fraction : Atomic fraction (at%) of the element
    """
    return (weight_percent[element] / atomic_weights[element]) / \
           sum(weight_percent[el] / atomic_weights[el] for el in weight_percent if el in atomic_weights)


def convert_atomic_to_weight_fraction(atomic_fraction_dict, atomic_weights, element):
    """
    Convert atomic fraction (at%) to weight percent (wt%).

    Input:
        atomic_fraction_dict : Dictionary containing atomic fractions {Element: at%}
        atomic_weights       : Dictionary containing atomic weights {Element: atomic weight}
        element              : The specific element to convert

    Output:
        weight_percent       : Weight percent (wt%) of the specified element
    """
 

    # Compute weight fraction using weight-to-atomic conversion for the denominator
    return (atomic_fraction_dict[element] * atomic_weights[element]) / \
           sum(convert_weight_to_atomic_fraction(atomic_fraction_dict, atomic_weights, el) * atomic_weights[el] 
               for el in atomic_fraction_dict if el in atomic_weights)


# Critical cross section is here just taken to be the mean cross section. It should be
# the mean cross section at peak at the ageing condition near peak strength.
def calculate_solid_solution_strength(alloy_composition, volume_fraction, atomic_weights, strengthening_coefficients):
    """
    Computes the solid solution strengthening contribution from alloying elements.

    Input:
        alloy_composition         : Dictionary containing element weight fractions {Element: wt%}
        volume_fraction           : Precipitate volume fraction (percentage, not decimal)
        atomic_weights            : Dictionary containing atomic weights {Element: atomic weight}
        strengthening_coefficients: Dictionary of strengthening coefficients {Element: MPa}

    Output:
        sigma_ss                  : Solid solution strengthening contribution [MPa]
    """

    # Step 1: Convert volume fraction to solid fraction using unit cell ratios (ρβ''/ρ_Al)
    solid_fraction = (volume_fraction / 100) * (1826 / 2005)
    print(f"Solid Fraction: {solid_fraction * 100:.4f}%")

    # Step 2: Atomic fractions in β'' (from Ninive et al.)
    betaDP = {
        'Mg': 0.353,
        'Al': 0.265,
        'Si': 0.353,
        'Cu': 0.029
    }

    # Step 3: Get atomic fraction of each element that precipitates out
    precipitated_atomic = {
        el: betaDP[el] * solid_fraction for el in ['Mg', 'Si', 'Cu']
    }

    # Step 4: Convert these atomic fractions to unnormalized weight %
    precipitated_weights = {
        el: precipitated_atomic[el] * atomic_weights[el] for el in precipitated_atomic
    }

    # Step 5: Subtract from original alloy composition
    weight_mg = max(0, alloy_composition['Mg'] - precipitated_weights['Mg'])
    weight_si = max(0, alloy_composition['Si'] - precipitated_weights['Si'])
    weight_cu = max(0, alloy_composition['Cu'] - precipitated_weights['Cu'])

    print(f"Weight % in Solution - Mg: {weight_mg:.3f}, Si: {weight_si:.3f}, Cu: {weight_cu:.3f}")

    # Step 6: Compute solid solution strengthening using power-law from literature
    sigma_ss = (
        strengthening_coefficients['Mg'] * weight_mg**(2/3) +
        strengthening_coefficients['Si'] * weight_si**(2/3) +
        strengthening_coefficients['Cu'] * weight_cu**(2/3)
    )

    return sigma_ss






def calculate_yield_strength(precipitate_lengths, mean_length,
                                    rho, aspect_ratio_params, critical_cross_section, 
                                    kappa, shear_modulus, burgers_vector, taylor_factor, 
                                    solid_solution_strength, base_strength,
                                    omega_func, shearable_integrand, non_shearable_integrand):
    """
    Computes the yield strength for a single alloy condition **without empirical calibration**.

    Input:
        precipitate_lengths      : List of precipitate lengths for the alloy condition [nm]
        mean_length              : Mean precipitate length in this condition [nm]
        rho                      : Number density of precipitates in this condition [#/nm³]
        aspect_ratio_params      : Parameters for the aspect ratio function Ω(l)
        critical_cross_section   : Critical cross-section a_c defining shearable/non-shearable transition [nm²]
        kappa                   : Scaling exponent for strength model
        shear_modulus            : Shear modulus G [MPa]
        burgers_vector           : Burgers vector b [nm]
        taylor_factor            : Taylor factor M (polycrystalline strengthening factor)
        solid_solution_strength  : Solid solution strengthening contribution [MPa]
        base_strength            : Baseline yield strength σ₀ [MPa]
        omega_func               : Function to compute aspect ratio Ω(l)
        shearable_integrand      : Function computing the integral for shearable precipitates
        non_shearable_integrand  : Function computing the integral for non-shearable precipitates

    Output:
        yield_strength           : Computed yield strength [MPa]
    """

    # Solve for the critical length l_c using least squares
    residual_func = lambda l: np.sqrt(critical_cross_section) * omega_func(aspect_ratio_params, l) - l
    critical_length = least_squares(residual_func, 16).x[0]  

    # Compute kernel bandwidth for KDE smoothing
    kernel_bandwidth = calculated_kernel_bandwidth(precipitate_lengths)

    # Define phi with proper arguments so it can be used inside quad()
    phi_function = lambda l: phi(l, precipitate_lengths, kernel_bandwidth)

    # Compute mean obstacle strength f_bar
    f_bar = (quad(shearable_integrand, 0, critical_length, args=(aspect_ratio_params, kappa, phi_function))[0] / (critical_cross_section**kappa) +
            quad(non_shearable_integrand, critical_length, 1000, args=(phi_function,))[0]) / mean_length

    # Compute number density of precipitate-based obstacles per slip plane
    obstacle_density = (np.sqrt(3) / 3) * mean_length * rho

    # Compute precipitate strengthening contribution σ_p
    sigma_p = taylor_factor * tau_p(alpha_p, shear_modulus, burgers_vector, obstacle_density, f_bar)

    # Debug print
    print(f"σ_p: {sigma_p:.2f} MPa, f_bar: {f_bar:.4f}, Obstacle density: {obstacle_density:.4f}")

    if sigma_p <= 0:
        raise ValueError(f"Error: σ_p = {sigma_p:.2f} MPa. Check f_bar ({f_bar:.4f}) and obstacle_density ({obstacle_density:.4f}).")

    # Compute final yield strength (no calibration)
    # May need to add some form of calibration
    yield_strength = sigma_p + solid_solution_strength + base_strength

    print(f"Yield Strength: {yield_strength:.2f} MPa")

    return yield_strength
    



In [170]:
# Compute aspect ratio parameters
aspect_ratio_params = fit_omega(precipitate_lengths, np.array(precipitate_lengths) / np.sqrt(mean_cross))

rho = convert_2D_to_3D_number_density(number_density, thickness_array, mean_length)
print('Number density [#/nm^3]: ', rho)
volume_fraction = calculate_volume_fraction(rho, mean_cross, mean_length)

alloy_compositions = {
    'var1': {'Mg': 0.700, 'Si': 0.850, 'Zn': 0.000, 'Cu': 0.100, 'Mn': 0.100, 'Fe': 0.150},
    'var2': {'Mg': 0.700, 'Si': 0.850, 'Zn': 0.000, 'Cu': 0.200, 'Mn': 0.100, 'Fe': 0.150},
    'var3': {'Mg': 0.700, 'Si': 0.850, 'Zn': 0.000, 'Cu': 0.350, 'Mn': 0.100, 'Fe': 0.150},
    'var4': {'Mg': 0.700, 'Si': 0.850, 'Zn': 0.000, 'Cu': 0.000, 'Mn': 0.250, 'Fe': 0.150},
    'var5': {'Mg': 0.700, 'Si': 0.850, 'Zn': 0.000, 'Cu': 0.200, 'Mn': 0.250, 'Fe': 0.150},
    'var6': {'Mg': 0.700, 'Si': 0.850, 'Zn': 0.000, 'Cu': 0.350, 'Mn': 0.250, 'Fe': 0.150},
    'var7': {'Mg': 0.700, 'Si': 0.850, 'Zn': 0.000, 'Cu': 0.350, 'Mn': 0.350, 'Fe': 0.150},
    'var8': {'Mg': 0.700, 'Si': 0.850, 'Zn': 0.000, 'Cu': 0.600, 'Mn': 0.250, 'Fe': 0.150}
}
# Found in SumAL
alloy_composition = alloy_compositions[dark_field_file[:4]]

atomic_weights = {'Al': 26.982, 'Mg': 24.305, 'Si': 28.085, 'Cu': 63.546, 'Fe': 55.845, 'Mn': 54.938, 'Cr': 51.996}
strengthening_coefficients = {'Mg': 29.0, 'Si': 66.3, 'Cu': 46.4} # Found in Literature(https://www.sciencedirect.com/science/article/pii/S0921509321017615?via%3Dihub)

# Compute solid solution strengthening

sigma_ss = calculate_solid_solution_strength(alloy_composition, volume_fraction, atomic_weights, strengthening_coefficients)
print(f"Solid Solution Strengthening: {sigma_ss:.2f} MPa")


# Compute solid solution strengthening

# Compute yield strength for a single condition
yield_strength = calculate_yield_strength(
    precipitate_lengths=precipitate_lengths,
    mean_length=mean_length,
    rho= rho,
    aspect_ratio_params=aspect_ratio_params,
    critical_cross_section= crit_cs,
    kappa=kappa,
    shear_modulus=G,
    burgers_vector=b,
    taylor_factor=M,
    solid_solution_strength=sigma_ss,
    base_strength=10,
    omega_func=omega,
    shearable_integrand=shearable_precipitate_integrand,
    non_shearable_integrand=non_shearable_precipitate_integrand
)

print(f"Yield Strength: {yield_strength:.2f} MPa")
hardness_hv = yield_strength / 3
print(f"Estimated Vickers Hardness (HV): {hardness_hv:.2f} MPa")



Number density [#/nm^3]: 

 8.246115777954009e-05
Solid Fraction: 0.0091%
Weight % in Solution - Mg: 0.699, Si: 0.849, Cu: 0.350
Solid Solution Strengthening: 105.33 MPa
σ_p: 290.32 MPa, f_bar: 1.0000, Obstacle density: 0.0009
Yield Strength: 405.65 MPa
Yield Strength: 405.65 MPa
Estimated Vickers Hardness (HV): 135.22 MPa
